# Arboles de Decisión


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from helpers import get_scores
from sklearn.grid_search import RandomizedSearchCV
from transformers import transformer
from data_builder import load_test_data, load_dev_data, load_small_dev_data

df, target = load_dev_data()
X = transformer.fit_transform(df)

print(X.shape)

(80993, 273)


# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.


Algunas observaciones previas:

El árbol que se genera no es usando ID3 sino CART. Una pequeña introducción
se puede ver en

http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart

No está muy completo el asunto ahí, habría que hurgar algún paper extra

Opciones:

- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [5]:
from search import find_best_classifier
from sklearn.ensemble import RandomForestClassifier

options = {
    'splitter': ['best', 'random'],
    'max_features': np.arange(0.1, 1.0, 0.1),
    'min_samples_split': range(2, 102, 10),
}

search_options = {
    'cv': 10,
    'scoring': 'roc_auc',
    'n_jobs': -1,
    'n_iter': 1000,
}

search = RandomizedSearchCV(DecisionTreeClassifier(), options, verbose=1, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 36.9min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 49.3min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 62.0min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 76.3min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 94.1min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 111.4min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 132.4min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 150.9min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 154.4min finished


Mejores parámetros: {'min_samples_split': 92, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 50, 'max_features': 0.59999999999999998}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
DecisionTreeClassifier,0.965289,0.967667,0.967749,0.970222,0.967667


# Haciendo PCA


Probemos ahora haciendo PCA sobre las variables, y luego aplicando el clasificador.

La reducción de dimensionalidad será una variable más del proceso de búsqueda de parámetros. Nos quedamos sólo con las variables de mayor componente principal. No sé si necesariamente es lo mejor esto...

In [6]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('pca', PCA()),
    ('tree', DecisionTreeClassifier()),
])
clf

Pipeline(steps=[('pca', PCA(copy=True, n_components=None, whiten=False)), ('tree', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])

Uso todas las opciones anteriores de árboles, pero tengo que agregarle un prefijo porque los meto en un pipeline (prefijo `tree__` para opciones del árbol, `pca__` para las de pca)

Voy a tomar como posibles dimensiones 20, 30, ..., n_dims - 10, n_dims

In [7]:
from helpers import add_prefix

new_options = add_prefix(options, 'tree__')

no_features = X.shape[1]

new_options.update({
    'pca__n_components' : range(20, no_features, 10)
})


search = RandomizedSearchCV(clf, new_options, verbose=1, **search_options)

search.fit(X, target)

print("Mejores parámetros: {}".format(search.best_params_))

get_scores(search.best_estimator_, transformer)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 30.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 59.9min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 100.6min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 144.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 195.1min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 252.2min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 318.0min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 391.7min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 472.0min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 573.2min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 670.3min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 768.9min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 785.2min finished


Mejores parámetros: {'tree__criterion': 'entropy', 'tree__splitter': 'best', 'tree__min_samples_split': 92, 'pca__n_components': 100, 'tree__max_features': 0.80000000000000004, 'tree__max_depth': 10}


,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
Pipeline,0.945659,0.948333,0.948488,0.951333,0.948333
